In [3]:
'''
对抗训练
1. adversarial training
2. adversarial fine-tuning
'''
import pandas as pd
import numpy as np
import json
from datasets import Dataset
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm
from sklearn.metrics import f1_score
import os
from torch.functional import F
from transformers import GPT2Tokenizer,GPT2ForSequenceClassification
from transformers import AdamW,get_linear_schedule_with_warmup
from datasets import load_dataset

/home/ljc/anaconda3/envs/adv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def adversari_training(model,tokenizer,train_data_path,valid_data_path,train_batch_size,eval_batch_size,num_epochs,lr,early_stopping,outputdir,trainlogdir,adv_training_data_path,training_or_finetuning = "training"):
    '''
    当training_or_finetuning == "fine-tuning"时,进行adversarial fine-tuning
    否则,进行adversarial training
    '''
    def tokenize_function(examples):
        return tokenizer(examples["text"],truncation = True,padding=True) 
    def collate_fn(examples):
        return tokenizer.pad(examples, padding="max_length", return_tensors="pt")
    traindataset = load_dataset("csv",data_files=train_data_path)["train"]
    validdataset = load_dataset("csv",data_files=valid_data_path)["train"]
    if training_or_finetuning == "training":
        advdataset = load_dataset("csv",data_files=adv_training_data_path)["train"]
        for i in advdataset:
            traindataset = traindataset.add_item(i)
            
    elif training_or_finetuning == "fine-tuning":
        advdataset = load_dataset("csv",data_files=adv_training_data_path)["train"]
        advdataset = advdataset.map(tokenize_function,batched=True,remove_columns=["text"]).rename_column("label","labels")
        adv_dataloader = DataLoader(advdataset , collate_fn=collate_fn,batch_size = train_batch_size)
    
    train_tokenized_dataset = traindataset.map(tokenize_function,batched=True,remove_columns=["text"]).rename_column("label","labels")
    valid_tokenized_dataset = validdataset.map(tokenize_function,batched=True,remove_columns=["text"]).rename_column("label","labels")
    train_dataloader = DataLoader(train_tokenized_dataset,shuffle=True,collate_fn=collate_fn,batch_size = train_batch_size)
    eval_dataloader = DataLoader(valid_tokenized_dataset , collate_fn=collate_fn,batch_size = eval_batch_size)
        
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)
    model.resize_token_embeddings(len(tokenizer))
    trainlogdf = pd.DataFrame(columns=["step","trainloss","validloss","acc","f1-score"])
    rowindex = 0
    eval_no_progress_count = 0
    epochloss = []
    max_eval_acc = 0
    optimizer = AdamW(params=model.parameters(), lr=lr)
    iter_to_accumlate = 4
    # Instantiate scheduler

    lr_scheduler = get_linear_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=0.06 * (len(train_dataloader) * num_epochs),
        num_training_steps=(len(train_dataloader) * num_epochs),
    )
    if training_or_finetuning == "fine-tuning":
        train_dataloader = adv_dataloader
        optimizer.load_state_dict(torch.load("checkpoint/optimizer.bin"))
        lr_scheduler.load_state_dict(torch.load("checkpoint/lr_scheduler.bin"))
    for epoch in range(num_epochs):
        model.train()
        allloss = 0
        for step,batch in enumerate(tqdm(train_dataloader)):
            batch.to(device)
            outputs = model(**batch)
            loss = outputs.loss/iter_to_accumlate
            loss.backward()
            allloss += loss.item()
            trainlogdf.loc[rowindex] = [rowindex,loss.item(),None,None,None]
            rowindex += 1
            epochloss.append(loss.item())
            if (step+1)%iter_to_accumlate==0:
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
            if (step+1)%(10*iter_to_accumlate) == 0:
                print("epoch",epoch,"step",step,"loss",loss,sep=" ")
        print("epoch",epoch,"step",step,"trainLoss:",allloss/(len(train_dataloader)*train_batch_size))
        
        count = 0
        model.eval()
        validloss = 0
        preds = []
        labels = []
        for evalstep,batch in enumerate(tqdm(eval_dataloader)):
            labels += batch['labels'].cpu()
            batch.to(device)
            with torch.no_grad():
                output = model(**batch)
            validloss += output.loss.item()
            pred = torch.argmax(F.softmax(output.logits.cpu(),dim=1),dim=1)
            preds += pred
            count += int(sum(batch['labels'].cpu() == pred))
        model.train()
        eval_acc = count/132
        trainlogdf.loc[rowindex-1,"validloss"] = validloss/132
        trainlogdf.loc[rowindex-1,"acc"] = eval_acc
        trainlogdf.loc[rowindex-1,"f1-score"] = f1_score(np.array(labels),np.array(preds),average="macro")
        print("epoch ",epoch,"step",step,"acc ",eval_acc)
        if eval_acc < max_eval_acc:
            eval_no_progress_count += 1
            if eval_no_progress_count >=early_stopping:
                print("Early Stopping:Epoch",epoch," Step",step,"Eval_acc",eval_acc,sep=" ")
                break
            else:
                print("Early Stopping record count",eval_no_progress_count,"Max eval acc",max_eval_acc,sep=" ")
        if eval_acc > max_eval_acc: 
            max_eval_acc = eval_acc
            print("Update Max eval acc",max_eval_acc)
            eval_no_progress_count = 0
            model.save_pretrained(outputdir)
            torch.save(model.state_dict(),os.path.join("checkpoint","model.bin"))
            torch.save(optimizer.state_dict(),os.path.join("checkpoint","optimizer.bin"))
            torch.save(lr_scheduler.state_dict(),os.path.join("checkpoint","lr_scheduler.bin"))
        
    trainlogdf.to_csv(trainlogdir)
    tokenizer.save_pretrained(outputdir)



In [3]:
model_path = "../../gpt2/"
model = GPT2ForSequenceClassification.from_pretrained(model_path,num_labels = 66)
tokenizer = GPT2Tokenizer.from_pretrained(model_path,bos_token = "<|startoftext|>",eos_token = "<|endoftext|>",pad_token = "<|pad|>",cls_token = "<|cls|>",sep_token = "<|sep|>" ,model_max_length = 1024)
adversari_training(model,tokenizer,"../dataset/data_folder/processed_gcjpy/train.csv","../dataset/data_folder/processed_gcjpy/valid.csv",2,2,30,5e-5,5,"GPT2-ADV-TRAINING","adv-training.log","../dataset/data_folder/processed_gcjpy/adv_training.csv","training")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at ../../gpt2/ and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Found cached dataset csv (/home/ljc/.cache/huggingface/datasets/csv/default-b125add5293cdc5e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 996.98it/s]
Found cached dataset csv (/home/ljc/.cache/huggingface/datasets/csv/default-ab07e3df6ac105f7/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 1248.30it/s]
Found cached dataset csv (/home/ljc/.cache/huggingface/datasets/csv/default-f03b36a028be774d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<0

epoch 0 step 39 loss tensor(1.6030, device='cuda:0', grad_fn=<DivBackward0>)


 15%|█▌        | 81/528 [00:13<01:11,  6.28it/s]

epoch 0 step 79 loss tensor(1.2158, device='cuda:0', grad_fn=<DivBackward0>)


 23%|██▎       | 121/528 [00:19<01:05,  6.26it/s]

...

epoch 22 step 519 loss tensor(0.0003, device='cuda:0', grad_fn=<DivBackward0>)


100%|██████████| 528/528 [01:24<00:00,  6.22it/s]


epoch 22 step 527 trainLoss: 0.0008856432745315859


100%|██████████| 66/66 [00:03<00:00, 18.13it/s]


epoch  22 step 527 acc  0.8863636363636364
Early Stopping:Epoch 22  Step 527 Eval_acc 0.8863636363636364


In [7]:
model = GPT2ForSequenceClassification.from_pretrained("GPT2saved_models")
tokenizer = GPT2Tokenizer.from_pretrained("GPT2saved_models")
adversari_training(model,tokenizer,"../dataset/data_folder/processed_gcjpy/train.csv","../dataset/data_folder/processed_gcjpy/valid.csv",2,2,30,5e-5,5,"GPT2-ADV-FINE-TUNING","adv-fine-tung.log","../dataset/data_folder/processed_gcjpy/adv_training.csv","fine-tuning")

Found cached dataset csv (/home/ljc/.cache/huggingface/datasets/csv/default-b125add5293cdc5e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 1151.33it/s]
Found cached dataset csv (/home/ljc/.cache/huggingface/datasets/csv/default-ab07e3df6ac105f7/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 1323.12it/s]
Found cached dataset csv (/home/ljc/.cache/huggingface/datasets/csv/default-f03b36a028be774d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 1264.87it/s]
/home/ljc/anaconda3/envs/adv/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 15%|█▌        | 40/264 [00:11<01:0

epoch 0 step 39 loss tensor(2.5713, device='cuda:0', grad_fn=<DivBackward0>)


 30%|███       | 80/264 [00:22<00:51,  3.57it/s]

epoch 0 step 79 loss tensor(1.3749, device='cuda:0', grad_fn=<DivBackward0>)


 45%|████▌     | 120/264 [00:33<00:41,  3.44it/s]

...

epoch 25 step 239 loss tensor(0.0012, device='cuda:0', grad_fn=<DivBackward0>)


100%|██████████| 264/264 [00:42<00:00,  6.23it/s]


epoch 25 step 263 trainLoss: 0.0014154510034572784


100%|██████████| 66/66 [00:03<00:00, 18.14it/s]

epoch  25 step 263 acc  0.9015151515151515
Early Stopping:Epoch 25  Step 263 Eval_acc 0.9015151515151515
